In [1]:
# import libraries
import glob                               #can be used to find the target files
import matplotlib.pyplot as plt
import nibabel as nib                   # to read and write neuron imaging data 
import numpy as np                        # for matrix calculating
import subprocess
from PIL import Image
import os,shutil

from scipy import signal
from skimage import color, img_as_ubyte, io

In [2]:
file_script = 'results_script1'
if not os.path.exists(file_script):
    os.mkdir(file_script)

CMD = 'shutil.copy(“fsltensor_V1.nii.gz”,”results_script1”)'
subprocess.call(CMD,shell=True)


In [3]:
cd results_script1

/home/zhiwei/Desktop/semester-projects/results_script1


## generate original tractography and get fib_original, hdr_original

In [4]:
data = nib.load('fsltensor_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask.nii')

In [5]:
CMD='tckgen fsltensor_V3.nii.gz origin0.015.tck -algorithm FACT -seed_image mask.nii -mask mask.nii -select 3000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [6]:
CMD = 'TractConverter.py -i origin0.015.tck -o origin0.015.trk -a fsltensor_FA.nii.gz'
subprocess.call(CMD,shell=True)

0

In [7]:
fib_original,hdr_original=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/origin0.015.trk')

## generate synthetic CLARITY data

In [8]:
CMD='tckmap origin0.015.tck syn_CLARITY.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0


## calculate structure tensor

In [9]:
def doggen(sigma=None, X=None, Y=None, Z=None):
    halfsize = np.ceil(3 * np.max(sigma))       #3*the maximum number of sequence sigma, ceil: shangxian
    x = np.arange(-halfsize,halfsize+1)         # sequence from -halfsize to halfsize+1

    if len(sigma) == 1:
        if X is None:
            X = x
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) )
    if len(sigma) == 2:
        if X is None or Y is None:
            [X,Y] =np.meshgrid(x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) )
    if len(sigma) == 3:
        if X is None or Y is None or Z is None:
            [X,Y,Z] =np.meshgrid(x,x,x)
        k = (-1)*X * np.exp( (-1)*np.power(X,2)/(2 * np.power(sigma[0],2)) ) * np.exp( (-1)*np.power(Y,2)/(2 * np.power(sigma[1],2)) ) * np.exp( (-1)*np.power(Z,2)/(2 * np.power(sigma[2],2)))
    if len(sigma) > 3:
        print ('Only support up to dimension 3')
    
    return k /np.sum(np.abs(k))

def gradCompute(img, dogsigma):
    dogkercc = doggen([dogsigma, dogsigma, dogsigma])
    dogkerrr = np.transpose( dogkercc, (1, 0, 2) )
    dogkerzz = np.transpose( dogkercc, (0, 2, 1) )

    gcc = signal.convolve(img, dogkercc, mode = 'valid')
    grr = signal.convolve(img, dogkerrr, mode = 'valid')
    gzz = signal.convolve(img, dogkerzz, mode = 'valid')

    # Gradient products
    gp = type('', (), {})()
    gp.gprrrr = grr * grr
    gp.gprrcc = grr * gcc
    gp.gprrzz = grr * gzz
    gp.gpcccc = gcc * gcc
    gp.gpcczz = gcc * gzz
    gp.gpzzzz = gzz * gzz

    # Gradient amplitude
    ga = np.sqrt(gp.gprrrr + gp.gpcccc + gp.gpzzzz)

    return ga, gp



## the optimal patch_size to retrieve the exact size of CLARITY as the originally synthetic one!! do not change this anymore!!

In [10]:
patch_size = 6.6

In [11]:

img_stack = nib.load('syn_CLARITY.nii.gz')
imgstack = img_stack.get_fdata()/255.0


# select sigma
dogsigma = [0.6]
vox_size = [15, 15, 15] #um
#define the voxel dimension to apply tensor
voxel_dim = np.array([patch_size,patch_size,patch_size]) # x,y,z in voxel dimension
volume_dim = (np.array(imgstack.shape)/voxel_dim).astype(np.int) # in voxel dimesion: removed voxel at the border

fsl_tensor = np.zeros((volume_dim[0], volume_dim[1], volume_dim[2], 6))
for x in range (0,volume_dim[0]):
    for y in range (0,volume_dim[1]):
        for z in range (0, volume_dim[2]):
            [ga,gp] = gradCompute(imgstack[int(x*voxel_dim[0]):int((x+1)*voxel_dim[0]), int(y*voxel_dim[1]):int((y+1)*voxel_dim[1]), int(z*voxel_dim[2]):int((z+1)*voxel_dim[2])], dogsigma)

            lowthreh = np.percentile(ga.reshape(-1), 20)
            highthreh = np.percentile(ga.reshape(-1), 80)

            mask = np.ones(ga.shape)
            mask[ga  < lowthreh] = 0
            mask[ga  > highthreh] = 0

            gprrrr = np.sum(gp.gprrrr[mask==1])
            gprrcc = np.sum(gp.gprrcc[mask==1])
            gprrzz = np.sum(gp.gprrzz[mask==1])
            gpcccc = np.sum(gp.gpcccc[mask==1])
            gpcczz = np.sum(gp.gpcczz[mask==1])
            gpzzzz = np.sum(gp.gpzzzz[mask==1])

            fsl_tensor[x,y,z] = np.array([[gprrrr, gprrcc, gprrzz,gpcccc, gpcczz, gpzzzz]])
affine = np.eye(4)
affine[0,0]=vox_size[0]*voxel_dim[0]*0.001
affine[1,1]=vox_size[1]*voxel_dim[1]*0.001
affine[2,2]=vox_size[2]*voxel_dim[2]*0.001
img_fsl_tensor = nib.Nifti1Image(fsl_tensor, affine)
nib.save(img_fsl_tensor, 'fsltensor_ftrial.nii.gz')
CMD='fsl5.0-fslmaths fsltensor_ftrial.nii.gz -tensor_decomp fsltensor_ftrial'
subprocess.call(CMD, shell=True)

/home/zhiwei/.local/lib/python2.7/site-packages/scipy/signal/signaltools.py:491: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return x[reverse].conj()


0

## generate the mask for subsequent tractography generation

In [12]:
data = nib.load('fsltensor_ftrial_FA.nii.gz')
d=np.ones(data.get_data().shape)
img=nib.Nifti1Image(d, data.affine)
nib.save(img,'mask_syn.nii')

##  generate the tractography and subsequent CLARITY based on the synthetic CLARITY images

In [13]:
number = 100000

In [14]:
CMD='tckgen fsltensor_ftrial_V3.nii.gz before0.015.tck -algorithm FACT -seed_image mask_syn.nii -mask mask_syn.nii -select 100000 -step 0.015'
subprocess.call(CMD, shell = True)

0

In [15]:
CMD = 'TractConverter.py -i before0.015.tck -o before0.015.trk -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [16]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/before0.015.trk')

In [17]:
CMD='tckmap before0.015.tck syn_CLARITY_before.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## function for comparison of two tractography on a voxel base

In [18]:
def compClarity(volume1,volume2):
    square_error = 0.0
    for x in range(volume1.shape[0]):
        for y in range(volume1.shape[1]):
            for z in range(volume1.shape[2]):
                square_error = square_error + np.square(volume1[x,y,z]-volume2[x,y,z])
    return square_error

## apply COMMIT to the generated tractography

In [19]:
path_commit = '/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT'
path = '/home/zhiwei/Desktop/semester-projects/results_script1'

In [20]:
trk_file_before = '/home/zhiwei/Desktop/semester-projects/results_script1/before0.015.trk'

In [21]:
from commit import trk2dictionary
trk2dictionary.run(filename_trk   = trk_file_before ,path_out= path_commit )
import commit
commit.core.setup()


-> Creating the dictionary from tractogram:
	* Segment position = COMPUTE INTERSECTIONS
	* Fiber shift X    = 0.000 (voxel-size units)
	* Fiber shift Y    = 0.000 (voxel-size units)
	* Fiber shift Z    = 0.000 (voxel-size units)
	* Points to skip   = 0
	* Min segment len  = 1.00e-03
	* Do not blur fibers
	* Loading data:
		* tractogram
			- 28 x 26 x 5
			- 0.0990 x 0.0990 x 0.0990
			- 100000 fibers
		* no mask specified to filter IC compartments
		* no dataset specified for EC compartments
		* output written to "/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT"
	* Generate tractogram matching the dictionary: 
	  [ 99781 fibers kept ]
   [ 12.1 seconds ]


In [22]:
mit=commit.Evaluation('.','results_script1')

In [23]:
mit.load_data('/home/zhiwei/Desktop/semester-projects/results_script1/syn_CLARITY.nii.gz','/home/zhiwei/Desktop/semester-projects/results_script1/BVECTOR.scheme')
mit.set_model('VolumeFractions')
hasISO=False
mit.model.set(hasISO)
mit.generate_kernels(regenerate=True)


-> Loading data:
	* DWI signal...
		- dim    = 191 x 178 x 39 x 1
		- pixdim = 0.015 x 0.015 x 0.015
	* Acquisition scheme...
		- 1 samples, 1 shells
		- 0 @ b=0 , 1 @ b=1000.0
   [ 0.0 seconds ]

-> Preprocessing:
	* There are no b0 volume(s) for normalization... [ min=0.00,  mean=0.25, max=28.00 ]
   [ 0.0 seconds ]

-> Simulating with "Volume fractions" model:
   [ 0.0 seconds ]


In [24]:
mit.load_kernels()
mit.load_dictionary(path_commit)
mit.set_threads(1)


-> Resampling LUT for subject "results_script1":
	* Keeping all b0 volume(s)... [ OK ]
	* Normalizing... [ OK ]
   [ 0.1 seconds ]

-> Loading the dictionary:
	* segments from the tracts... [ 99781 fibers and 4812915 segments ]
	* segments from the peaks...  [ 0 segments ]
	* isotropic contributions...  [ 3580 voxels ]
	* post-processing...          [ OK ]
   [ 1.2 seconds ]

-> Distributing workload to different threads:
	* number of threads : 1
	* A operator...  [ OK ]
	* A' operator... [ OK ]
   [ 0.0 seconds ]


In [25]:
mit.build_operator()


-> Building linear operator A:
   [ 3.3 seconds ]


In [26]:
mit.fit()
## mit.fit(tol_fun = 1e-7, max_iter = 100000000)


-> Fit model

      |     ||Ax-y||     |  Cost function    Abs error      Rel error    |     Abs x          Rel x
------|------------------|-----------------------------------------------|------------------------------
   1  |   3.2653759e+03  |  5.3313400e+06  3.6294864e+07  6.8078314e+00  |  1.9188958e+02  1.1041560e+00
   2  |   2.1698470e+03  |  2.3541180e+06  2.9772220e+06  1.2646868e+00  |  5.1131140e+01  3.8653859e-01
   3  |   1.4342165e+03  |  1.0284885e+06  1.3256295e+06  1.2889103e+00  |  3.8427962e+01  3.7823379e-01
   4  |   9.5830984e+02  |  4.5917888e+05  5.6930965e+05  1.2398429e+00  |  2.8214561e+01  3.5385482e-01
   5  |   6.6138736e+02  |  2.1871662e+05  2.4046226e+05  1.0994238e+00  |  2.0333924e+01  3.1642346e-01
   6  |   4.7345064e+02  |  1.1207775e+05  1.0663887e+05  9.5147220e-01  |  1.4949392e+01  2.8182975e-01
   7  |   3.4950447e+02  |  6.1076688e+04  5.1001065e+04  8.3503325e-01  |  1.1297714e+01  2.5277341e-01
   8  |   2.6526835e+02  |  3.5183649e+04  2.

  77  |   1.4288861e+01  |  1.0208577e+02  2.3090673e-01  2.2618895e-03  |  7.5634222e-02  1.9771026e-02
  78  |   1.4273474e+01  |  1.0186603e+02  2.1974143e-01  2.1571610e-03  |  7.3221637e-02  1.9116671e-02
  79  |   1.4258830e+01  |  1.0165712e+02  2.0891089e-01  2.0550543e-03  |  7.1359849e-02  1.8602832e-02
  80  |   1.4244875e+01  |  1.0145823e+02  1.9889101e-01  1.9603241e-03  |  6.8506808e-02  1.7828408e-02
  81  |   1.4231490e+01  |  1.0126765e+02  1.9057214e-01  1.8818659e-03  |  6.6989688e-02  1.7400177e-02
  82  |   1.4218658e+01  |  1.0108511e+02  1.8253987e-01  1.8058036e-03  |  6.5894478e-02  1.7079738e-02
  83  |   1.4206320e+01  |  1.0090977e+02  1.7534665e-01  1.7376579e-03  |  6.4844387e-02  1.6769337e-02
  84  |   1.4194384e+01  |  1.0074028e+02  1.6949227e-01  1.6824678e-03  |  6.3923882e-02  1.6491009e-02
  85  |   1.4182930e+01  |  1.0057775e+02  1.6252881e-01  1.6159520e-03  |  6.3048850e-02  1.6223140e-02
  86  |   1.4171863e+01  |  1.0042085e+02  1.5689595e-0

In [27]:
mit.save_results(save_coeff=True)


-> Saving results to "Results_VolumeFractions/*":
	* configuration and results:
		- pickle...  [ OK ]
		- txt...  [ OK ]
	* fitting errors:
		- RMSE...  [ 0.075 +/- 0.223 ]
		- NRMSE... [ 0.018 +/- 0.113 ]
	* voxelwise contributions:
		- intra-axonal [ OK ]
		- extra-axonal [ OK ]
		- isotropic    [ OK ]
   [ 0.6 seconds ]


In [28]:
trk_file = os.path.join(path_commit,'dictionary_TRK_fibers.trk')
fib_tmp,trk_hdr_tmp=nib.trackvis.read(trk_file)

## generate tractography after applying COMMIT with weight > 0, and get fib_after,hdr_after

In [29]:
f = open('/home/zhiwei/Desktop/semester-projects/results_script1/COMMIT/Results_VolumeFractions/xic.txt')


In [30]:
data = f.read().splitlines()
data = map(eval,data)
j=0
for items in range(len(fib_tmp)-1,-1,-1):
    if data[j] == 0:
        del fib_tmp[items]
    j=j+1
nib.trackvis.write('fibers_after.trk',fib_tmp,trk_hdr_tmp)

In [31]:
fib_after,hdr_after=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_after.trk')

In [32]:
CMD = 'TractConverter.py -i fibers_after.trk -o fibers_after.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [33]:
CMD='tckmap fibers_after.tck syn_CLARITY_after.nii.gz -vox 0.015'
subprocess.call(CMD, shell = True)

0

## downsample fibers(before/after) to the original one, take mean distance and std as measurement

## downsample the fibers and get tracks_before, tracks_after, tracks_original

In [34]:
from dipy.tracking.metrics import downsample
import random
tracks_after = [downsample(fib_after[i][0],7) for i in range(len(fib_after))]
tracks_original = [downsample(fib_original[i][0],7) for i in range(len(fib_original))]
## generate a random list to get equal number of fibers from fib before COMMIT
list = range(len(fib_before))
slice1 = random.sample(list, len(fib_after))
tracks_before1 = [downsample(fib_before[i][0],7) for i in slice1]

In [35]:
from dipy.segment.bundles import bundles_distances_mdf
DM_before1 = bundles_distances_mdf(tracks_before1, tracks_original)
minD_before1 = [min(DM_before1[i,:]) for i in range(len(DM_before1))]
DM_after = bundles_distances_mdf(tracks_after, tracks_original)
minD_after = [min(DM_after[i,:]) for i in range(len(DM_after))]

## compare tracks_after and tracks_before with tracks_original with mean distance and STD

In [36]:
meanD_before= np.mean(minD_before1)
stdD_before= np.std(minD_before1)

meanD_after= np.mean(minD_after)
stdD_after= np.std(minD_after)

In [37]:
meanD_after

0.1909169839439658

In [38]:
meanD_before

0.1910786749077243

meanD_after

## Compare on a voxel level

original resolution -- 0.015mm

In [39]:
for items in range(len(fib_before)-1,-1,-1):
    if items not in slice1:
        del fib_before[items]
    j=j+1
nib.trackvis.write('fibers_before.trk',fib_before,hdr_before)

In [40]:
fib_before,hdr_before=nib.trackvis.read('/home/zhiwei/Desktop/semester-projects/results_script1/fibers_before.trk')

In [41]:
CMD = 'TractConverter.py -i fibers_before.trk -o fibers_before.tck -a fsltensor_ftrial_FA.nii.gz'
subprocess.call(CMD,shell = True)

0

In [42]:
CMD='tckmap fibers_before.tck syn_CLARITY_random.nii.gz -vox 0.015' 
subprocess.call(CMD, shell = True)

0

In [43]:
data_original = nib.load('syn_CLARITY.nii.gz')
data_before = nib.load('syn_CLARITY_random.nii.gz')
data_after = nib.load('syn_CLARITY_after.nii.gz')
## normalization
sum_original = sum(sum(sum(data_original.get_fdata())))
sum_before = sum(sum(sum(data_before.get_fdata())))
sum_after = sum(sum(sum(data_after.get_fdata())))
volume_original = data_original.get_fdata()/sum_original
volume_before = data_before.get_fdata()/sum_before
volume_after = data_after.get_fdata()/sum_after

In [44]:
square_error_before = compClarity(volume_before, volume_original)
square_error_after = compClarity(volume_after, volume_original)

## change the resolution here

resolution based on mean distance after COMMIT

In [45]:
CMD='tckmap origin0.015.tck syn_CLARITY_COMMIT_res.nii.gz -vox 0.193' 
subprocess.call(CMD, shell = True)

0

In [46]:
CMD='tckmap fibers_after.tck syn_CLARITY_after_COMMIT_res.nii.gz -vox 0.193'
subprocess.call(CMD, shell = True)

0

In [47]:
CMD='tckmap fibers_before.tck syn_CLARITY_random_COMMIT_res.nii.gz -vox 0.193'
subprocess.call(CMD, shell = True)

0

In [48]:
data_original_COMMIT_res = nib.load('syn_CLARITY_COMMIT_res.nii.gz')
data_before_COMMIT_res = nib.load('syn_CLARITY_random_COMMIT_res.nii.gz')
data_after_COMMIT_res = nib.load('syn_CLARITY_after_COMMIT_res.nii.gz')
## normalization
sum_original_COMMIT_res = sum(sum(sum(data_original_COMMIT_res.get_fdata())))
sum_before_COMMIT_res = sum(sum(sum(data_before_COMMIT_res.get_fdata())))
sum_after_COMMIT_res = sum(sum(sum(data_after_COMMIT_res.get_fdata())))
volume_original_COMMIT_res = data_original_COMMIT_res.get_fdata()/sum_original_COMMIT_res
volume_before_COMMIT_res = data_before_COMMIT_res.get_fdata()/sum_before_COMMIT_res
volume_after_COMMIT_res = data_after_COMMIT_res.get_fdata()/sum_after_COMMIT_res

In [49]:
square_error_before_COMMIT_res = compClarity(volume_before_COMMIT_res, volume_original_COMMIT_res)
square_error_after_COMMIT_res = compClarity(volume_after_COMMIT_res, volume_original_COMMIT_res)

## Compare on a voxel level - binary

original resolution -- 0.015mm

In [50]:
def binarize(volume):
    volume_prime = np.zeros([len(volume),len(volume[1]),len(volume[1][1])])
    for x in range(len(volume)):
        for y in range(len(volume[1])):
            for z in range(len(volume[1][1])):
                if volume[x,y,z]!=0:
                    volume_prime[x,y,z]=1
    return volume_prime

In [51]:
Volume_Original = binarize(volume_original)
Volume_Before = binarize(volume_before)
Volume_After = binarize(volume_after)
error_before = compClarity(Volume_Before, Volume_Original)
error_after = compClarity(Volume_After, Volume_Original)

## change the resolution here

COMMIT resolution 0.186

In [52]:
Volume_Original_COMMIT_res = binarize(volume_original_COMMIT_res)
Volume_Before_COMMIT_res = binarize(volume_before_COMMIT_res)
Volume_After_COMMIT_res = binarize(volume_after_COMMIT_res)
error_before_COMMIT_res = compClarity(Volume_Before_COMMIT_res, Volume_Original_COMMIT_res)
error_after_COMMIT_res = compClarity(Volume_After_COMMIT_res, Volume_Original_COMMIT_res)

## save results

In [53]:
cd ..

/home/zhiwei/Desktop/semester-projects


In [54]:
f = open('meanD_std3000.txt','a')
f.write('group: Nboffibers_before: ' + str(number))
f.write('         Nboffibers_after: ' + str(len(fib_after)))
f.write('\n\ncompare tractography on a tract level:')
#f.write('\n1. mean distance before COMMIT:  ' + str(meanD_before1))
#f.write('\n   STD before COMMIT:            ' + str(stdD_before1))
#f.write('\n2. mean distance before COMMIT:  ' + str(meanD_before2))
#f.write('\n   STD before COMMIT:            ' + str(stdD_before2))
#f.write('\n3. mean distance before COMMIT:  ' + str(meanD_before3))
#f.write('\n   STD before COMMIT:            ' + str(stdD_before3))
#f.write('\n')
    
f.write('\n2. mean distance random:    ' + str(meanD_before))
f.write('\n   STD random:              ' + str(stdD_before)) 

f.write('\n1. mean distance COMMIT:     ' + str(meanD_after))
f.write('\n   STD COMMIT:               ' + str(stdD_after)) 

f.write('\n\n')
f.close()

In [55]:
f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level - BINARIZATION:')
f.write('\nerror random: ' + str(error_before))
f.write('\nerror COMMIT: ' + str(error_after)) 
f.write('\n\n')
f.close()

In [56]:
f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level - non-BINARIZATION:')
f.write('\nsquare error random: ' + str(square_error_before))
f.write('\nsquare error COMMIT: ' + str(square_error_after)) 
f.write('\n\n\n\n\n')
f.close()

In [57]:
f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res - BINARIZATION:')
f.write('\nerror random: ' + str(error_before_COMMIT_res))
f.write('\nerror COMMIT: ' + str(error_after_COMMIT_res)) 
f.write('\n\n')
f.close()

In [58]:
f = open('meanD_std3000.txt','a')
f.write('\n')
f.write('compare tractography on a voxel level_COMMIT_res- non-BINARIZATION:')
f.write('\nsquare error random: ' + str(square_error_before_COMMIT_res))
f.write('\nsquare error COMMIT: ' + str(square_error_after_COMMIT_res)) 
f.write('\n\n\n\n\n')
f.close()